## Imports

In [1]:
import os
import cv2
import pickle
import imageio
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.use('Agg')
from matplotlib.animation import FFMpegWriter
from tqdm import tqdm
from pprint import pprint
import sys
sys.path.append('./')
from utils import normalize_image, get_sequences

In [2]:
from enum import Enum
from collections import namedtuple

class ModelType(Enum):
    CLASSIFICATION = 0
    SEGMENTATION = 1

classlabels_viz_colors = ['black', 'green', 'yellow', 'blue', 'red', 'magenta', 'cyan',
                          'lightseagreen', 'brown', 'magenta', 'olive', 'wheat', 'white', 'black']
classlabels_viz_bounds = [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 100]

classlabels_viz_cmap = mpl.colors.ListedColormap(classlabels_viz_colors)
classlabels_viz_norm = mpl.colors.BoundaryNorm(classlabels_viz_bounds, classlabels_viz_cmap.N)

confidence_heatmap_viz_colors = ['black', 'blue', 'red', 'orange', 'yellow', 'lightgreen', 'lightseagreen']
confidence_heatmap_viz_bounds = [-1, 0,0.5,0.6,0.7,0.8,0.9,1]
confidence_heatmap_viz_cmap = mpl.colors.ListedColormap(confidence_heatmap_viz_colors)
confidence_heatmap_viz_norm = mpl.colors.BoundaryNorm(confidence_heatmap_viz_bounds, confidence_heatmap_viz_cmap.N)


LabelColor = namedtuple('LabelColor', ['name', 'id', 'trainid', 'color', 'category'])

LABEL_COLORS = [
    LabelColor('class1', 1, 0, (128, 0, 128), 'driveableterrain'),
    LabelColor('class2', 2, 1, (255, 0, 0), 'non-driveableterrain'),
    LabelColor('class3', 3, 2, (0, 0, 255), 'sky'),
    LabelColor('class4', 4, 3, (0, 255, 0), 'trees'),
    LabelColor('class5', 5, 4, (255, 0, 255), 'implement'),
    LabelColor('class6', 6, 5, (255, 255, 0), 'basket markers')
]

LABEL_COLORS_4CLASS = LABEL_COLORS[0:4]
LABEL_COLORS_5CLASS = LABEL_COLORS[0:5]
LABEL_COLORS_6CLASS = LABEL_COLORS[0:6]
LABEL_COLORS_SKY_DET = [LABEL_COLORS[0], LABEL_COLORS[2]]

LABEL_COLORS_IMPL = [
    LabelColor('class1', 1, 1, (128, 0, 128), 'implement'),
    LabelColor('class2', 2, 2, (255, 0, 0), 'sweep'),
    LabelColor('class3', 3, 3, (0, 0, 255), 'harrow_tine'),
    LabelColor('class4', 4, 4, (0, 255, 0), 'basket'),
    LabelColor('class5', 5, 5, (0, 255, 0), 'basket_marker'),
    LabelColor('class6', 0, 255, (0, 0, 0), 'ignore')
]

LABEL_COLORS_IMPL_REDUCED = [
    LabelColor('class0', 1, 0, (0, 0, 0), 'background'),
    LabelColor('class1', 2, 1, (0, 255, 0), 'implement'),
    LabelColor('class2', 3, 2, (255, 0, 0), 'sweep'),
    LabelColor('class3', 4, 3, (0, 255, 0), 'basket_marker'),
    LabelColor('class6', 0, 255, (0, 0, 0), 'ignore')
]

PLUG_LABEL_MAP ={0: 'no-plug', 1: 'plug'}
IMPL_SEGMENT_LABEL_MAP = {0: 'background', 1: 'implement', 2: 'sweep', 3:'basket_marker'}

In [3]:
def create_mpl_viz_outputs(output_path,
                           image,
                           prediction_labels,
                           confidences,
                           depth_img, 
                           image_title='Image', 
                           pred_title='Prediction', 
                           conf_title='Confidence', 
                           depth_title='Depth',
                           bbox_coords=[]):
    """
    Utility function to plot results based on order of input provided.

    Axes index can have different values based on input provided.

    For example: If image, prediction and groundtruth_label is provided, A three axes plot will be generated with
    image(ax1), ground_truth(ax2), prediction(ax3).

    Order of plot if all the inputs are provided will be in same order as arguments listed above.
    """
    axis_index = list(range(len(list(filter(lambda x: x is not None, [image,
                                                                      prediction_labels, confidences,
                                                                      depth_img])))))
    axis_curr_index = 0
    fig, axes = mpl.pyplot.subplots(1, len(axis_index), figsize=((60, 30)))
    if bbox_coords:
#         xmin, ymin, xmax, ymax = bbox_coords
        xmin, xmax, ymin, ymax = bbox_coords
        if (xmin < 0) or (ymin < 0):
            raise ValueError(f'Either {xmin} or {ymin} are negative')
        else:
            rect = mpl.patches.Rectangle((ymin, xmin), (ymax - ymin), (xmax - xmin), linewidth=3, edgecolor='k',
                                     facecolor='none')
    else:
        rect = None

    if axis_curr_index < len(axes):
        # Grab only RGB channels from image, otherwise depth with distort the image when it is displayed
        axes[axis_curr_index].imshow(image)
        axes[axis_curr_index].set_title(image_title, fontsize=30)
        axes[axis_curr_index].axis('off')
        axis_curr_index += 1

    if axis_curr_index < len(axes):
        axes[axis_curr_index].imshow(depth_img, cmap='turbo')
        axes[axis_curr_index].set_title(depth_title, fontsize=30)
        axes[axis_curr_index].axis('off')
        axis_curr_index += 1

    if axis_curr_index < len(axes):
        axes[axis_curr_index].imshow(prediction_labels, classlabels_viz_cmap, classlabels_viz_norm, interpolation='nearest')
        if rect is not None:
            rect1 = mpl.patches.Rectangle((ymin, xmin), (ymax - ymin), (xmax - xmin), linewidth=3, edgecolor='k',facecolor='none')
            axes[axis_curr_index].add_patch(rect1)
        axes[axis_curr_index].set_title(pred_title, fontsize=30)
        axes[axis_curr_index].axis('off')
        axis_curr_index += 1

    if axis_curr_index < len(axes):
        c = np.max(confidences, axis=2)
        axes[axis_curr_index].imshow(c, confidence_heatmap_viz_cmap, confidence_heatmap_viz_norm, interpolation='nearest')
#         if rect is not None:
#             rect2 = mpl.patches.Rectangle((ymin, xmin), (ymax - ymin), (xmax - xmin), linewidth=3, edgecolor='k',
#                                           facecolor='none')
#             axes[axis_curr_index].add_patch(rect2)
        axes[axis_curr_index].set_title(conf_title, fontsize=30)
        axes[axis_curr_index].axis('off')
        axis_curr_index += 1

    mpl.pyplot.savefig(output_path, pad_inches=0, bbox_inches='tight', dpi=150)
    mpl.pyplot.close('all')

def read_image(image_path):
    image = (np.load(image_path) * 255).astype(np.uint8)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image

def read_saved_frame(pred_dir, image_id):
    states_to_save = ['', 'false_positive', 'false_negative', 'large_object_false_negative', 'true_positive', 'true_negative']
    frame = None
    for state in states_to_save:
        if os.path.isfile(os.path.join(pred_dir, state, image_id+'.png')):
            frame = cv2.imread(os.path.join(pred_dir, state, image_id+'.png'))
            break
        if os.path.isfile(os.path.join(pred_dir, state, image_id+'.jpg')):
            frame = cv2.imread(os.path.join(pred_dir, state, image_id+'.jpg'))
            break
    return frame

def read_images(pred_dir, _id):
    if not os.path.isfile(os.path.join(pred_dir, _id+'_image.npy')):
        return None, None, None, None
    image = np.load(os.path.join(pred_dir, _id+'_image.npy'))
    
    # 100m capped depth
    depth = np.load(os.path.join(pred_dir, _id+'_depth.npy'))
#     # raw depth
#     raw_depth_dir = '/raum_raid/li.yu/data/Jupiter_rock_demo_2021/Jupiter_rock_demo_loamy06_Oct20_2021/model_processed_v4.1_sky_2e-3_lr_1e-3_color_aug_full_model_LR_consistency_regularization_0.2_epoch_23/images/'
#     stereo_data = np.load(os.path.join(raw_depth_dir, _id, 'stereo_output.npz'))
#     depth = stereo_data['point_cloud'][:,:,-1]
#     depth = normalize_and_clip_depth(depth, 200)
    
    pred_label = np.load(os.path.join(pred_dir, _id+'_pred_label.npy'))
    confidence = np.load(os.path.join(pred_dir, _id+'_confidence.npy'))
    return image, depth, pred_label, confidence

# def create_frame(pred_dir, pred_merged_dir, _id, recreate=False):
#     canvas_path = os.path.join(pred_merged_dir, _id+'.png')
#     if recreate:
#         image, depth, pred_label, confidence = read_images(pred_dir, _id)
#         if image is None:
#             return None
#         create_mpl_viz_outputs(canvas_path, image, pred_label, confidence, depth)
#     frame = cv2.imread(canvas_path)
#     return frame

def get_bbox_coords(i=-1, bbox_range_list=[], bbox_coord_list=[]):
    for bi in range(len(bbox_range_list)):
        bbox_range = bbox_range_list[bi]
        if bbox_range[0] <= i <= bbox_range[1]:
            return bbox_coord_list[bi]
    return []

def process_frame(pred_dir, pred_merged_dir, _id, recreate=False, bbox_coords=[]):
    image, depth, pred_label, confidence = None, None, None, None
    l = 0.0
    avg_pixel = 0.0
    bbox_conf = None
    if recreate:
        image, depth, pred_label, confidence = read_images(pred_dir, _id)
        if image is None:
            return image, depth, pred_label, confidence, l, avg_pixel, bbox_conf
        # calculate brightness
        hlsImg = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
        l = np.average(hlsImg[:,:,1])
        image_title = 'Image (brightness: {:.4f})'.format(l)
        # calculate average pixel value at bbox area
        if bbox_coords:
            ymin, ymax, xmin, xmax = bbox_coords
            bbox_pred = pred_label[ymin:ymax+1, xmin:xmax+1]
            bbox_conf = confidence[ymin:ymax+1, xmin:xmax+1]
            avg_pixel = np.count_nonzero(bbox_pred == 1)
    return image, depth, pred_label, confidence, l, avg_pixel, bbox_conf

def create_frame(pred_dir, pred_merged_dir, _id, recreate=False, bbox_coords=[]):
    canvas_path = os.path.join(pred_merged_dir, _id+'.png')
    image, depth, pred_label, confidence = None, None, None, None
    l = 0.0
    avg_pixel = 0.0
    bbox_conf = None
    if recreate:
        image, depth, pred_label, confidence = read_images(pred_dir, _id)
        if image is None:
            return None, None, None, None, None, None, None, None
        # calculate brightness
        hlsImg = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
        l = np.average(hlsImg[:,:,1])
        image_title = 'Image (brightness: {:.4f})'.format(l)
        # calculate average pixel value at bbox area
        if bbox_coords:
            ymin, ymax, xmin, xmax = bbox_coords
            bbox_pred = pred_label[ymin:ymax+1, xmin:xmax+1]
            bbox_conf = confidence[ymin:ymax+1, xmin:xmax+1]
            avg_pixel = np.count_nonzero(bbox_pred == 1)
        create_mpl_viz_outputs(canvas_path, image, pred_label, confidence, depth, image_title=image_title, bbox_coords=bbox_coords)
    frame = cv2.imread(canvas_path)
    return frame, image, depth, pred_label, confidence, l, avg_pixel, bbox_conf

def create_diff_frame(pred_merged_dir, _id, image1, depth1, pred_label1, confidence1, 
                      image2, depth2, pred_label2, confidence2, pred_title='prediction', conf_title='confidence'):
    canvas_path = os.path.join(pred_merged_dir, _id+'_diff.png')
    image = np.abs(image1 - image2)
    depth = np.abs(depth1 - depth2)
    pred_label = np.abs(pred_label1 - pred_label2)
    confidence = np.abs(confidence1 - confidence2)
    create_mpl_viz_outputs(canvas_path, image, pred_label, confidence, depth, conf_title=conf_title)
    frame = cv2.imread(canvas_path)
    return frame

def read_raw_image_by_id(data_dir, _id):
    image_path = os.path.join(data_dir, 'images', _id, 'artifact_debayeredrgb_0_'+_id+'.png')
    image = cv2.imread(image_path)
    return image

def read_raw_image_by_row(data_dir, row):
    image = cv2.imread(os.path.join(data_dir, row.artifact_debayeredrgb_0_save_path))
    return image

def create_video(ids, pred_dir, video_name, read_func=read_saved_frame, fps=2):
    frame = read_func(pred_dir, ids[10])
    height, width, layers = frame.shape
    print(height, width, layers)

    # .avi MJPG,  .mp4 MP4V
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), fps, (width,height), isColor=True)
    
    good = 0
    for _id in tqdm(ids):
        frame = read_func(pred_dir, _id)
        if frame is not None:
            video.write(frame)
            good += 1
    print('total', len(ids), 'used', good)

    # cv2.destroyAllWindows()
    video.release()

## Create video from raw left images

In [8]:
# create video from raw left images
# data_dir = '/data/jupiter/datasets/halo_failure_case_of_box_in_dust'
# data_dir = '/data/jupiter/datasets/halo_missed_lo_rock_0509_stereo'
# data_dir = '/data/jupiter/li.yu/data/halo_sample_terrains_15images'
# data_dir = '/data/jupiter/li.yu/data/halo_sample_terrains_15images_all_camera_sequence'
# data_dir = '/data/jupiter/datasets/halo_human_in_dust_day_collection_may29'
# data_dir = '/data/jupiter/datasets/halo_human_in_dust_night_collection_june03'
# data_dir = '/data/jupiter/datasets/halo_human_in_dust_night_collection_june03_2'
# data_dir = '/data/jupiter/datasets/halo_human_in_dust_day_collection_back_june05'
# data_dir = '/data/jupiter/datasets/halo_human_in_dust_dusk_collection_front_june07'
# data_dir = '/data/jupiter/datasets/halo_vehicles_in_dust_collection_march2024'
# data_dir = '/data/jupiter/datasets/halo_vehicles_in_dust_collection_june06'
# data_dir = '/data/jupiter/datasets/halo_vehicles_in_dust_collection_june07'
# data_dir = '/data/jupiter/datasets/halo_vehicles_in_dust_collection_june04'
# data_dir = '/data/jupiter/datasets/halo_dust_on_lens_blur_dataset_v3_20240807'
# data_dir = '/data/jupiter/datasets/dust_datasets/halo_dust_on_lens_blur_dataset_v3_20240807'
data_dir = '/data/jupiter/datasets/dust_datasets/Jupiter_bedrock_40013_20240617_dust_sequences'
# data_dir = '/data/jupiter/datasets/dust_datasets/halo_human_in_dust_night_collection_july29'
df = pd.read_csv(os.path.join(data_dir, 'annotations_full.csv'))
# df = pd.read_csv(os.path.join(data_dir, 'master_annotations.csv'))

seq_dfs = get_sequences(df, interval=10, per_camera_pair=False)  # break the data by intervals between sequences
print(df.shape, len(seq_dfs))

(92493, 107) 11


In [43]:
df.iloc[0].collected_on, df.iloc[-1].collected_on

('2024-06-17T21:23:50.296000', '2024-06-17T21:24:49.903000')

### select images for labeling

In [31]:
# combine and create datasets for human in dust
data_root_dir = '/data/jupiter/datasets'
unlabeled_datasets = [
    "halo_human_in_dust_day_collection_may29",
    "halo_human_in_dust_night_collection_june03",  # for lying down humans
    "halo_human_in_dust_night_collection_june03_2",
    "halo_human_in_dust_day_collection_back_june05",
    "halo_human_in_dust_dusk_collection_front_june07",
]
day_night_splits = ['day', 'dawn_dusk', {'dawn_dusk': list(range(0, 9)), 'night': list(range(9, 19))}, 'day', 'dawn_dusk']
use_ds = [0, 2, 3, 4]
# split data by day time
image_ids_by_time = {'id':[], 'day_night_split': []}
for di in use_ds:
    dataset = unlabeled_datasets[di]
    df = pd.read_csv(os.path.join(data_root_dir, dataset, 'annotations.csv'))
    df = df.sort_values('collected_on')
    day_night_split = day_night_splits[di]
    if isinstance(day_night_split, str):
        print(dataset, df.shape, day_night_split)
        image_ids_by_time['id'] += df.id.to_list()
        image_ids_by_time['day_night_split'] += [day_night_split] * len(df)
    else:
        seq_dfs = get_sequences(df, interval=60)  # break the data by intervals between sequences
        print(dataset, df.shape, len(seq_dfs), day_night_split)
        for dn_split, seq_ids in day_night_split.items():
            for seq_id in seq_ids:
                seq_df = seq_dfs[seq_id]
                image_ids_by_time['id'] += seq_df.id.to_list()
                image_ids_by_time['day_night_split'] += [dn_split] * len(seq_df)
# save to csv
day_night_split_df = pd.DataFrame(data=image_ids_by_time)
print(day_night_split_df.shape)
day_night_split_df.to_csv('/data/jupiter/li.yu/data/halo_hard_cases/halo_human_in_dust_daynightsplit_for_july01.csv')

halo_human_in_dust_day_collection_may29 (48102, 108) day
halo_human_in_dust_night_collection_june03_2 (30284, 109) 19 {'dawn_dusk': [0, 1, 2, 3, 4, 5, 6, 7, 8], 'night': [9, 10, 11, 12, 13, 14, 15, 16, 17, 18]}
halo_human_in_dust_day_collection_back_june05 (11675, 108) day
halo_human_in_dust_dusk_collection_front_june07 (36338, 104) dawn_dusk
(126399, 2)


In [27]:
# image_ids_by_time = {'id':[], 'day_night_split': []}
# split data in day/dusk/night
day_night_splits = [{'day': list(range(0,25)), 'dawn_dusk': list(range(25, 31)), 'night': list(range(31, 38))}, 
                   {'dawn_dusk': list(range(0, 14)), 'night': list(range(14, 29))},
                   {'dawn_dusk': [0, 1, 2, 7, 8, 9], 'night': [3, 4, 5, 6, 10, 11, 12]}]
day_night_split = day_night_splits[2]
for dn_split, seq_ids in day_night_split.items():
    for seq_id in seq_ids:
        seq_df = seq_dfs[seq_id]
        image_ids_by_time['id'] += seq_df.id.to_list()
        image_ids_by_time['day_night_split'] += [dn_split] * len(seq_df)
# save to csv
day_night_split_df = pd.DataFrame(data=image_ids_by_time)
print(day_night_split_df.shape)
day_night_split_df.to_csv('/data/jupiter/li.yu/data/halo_hard_cases/halo_vehicle_in_dust_daynightsplit_excludebadnight_for_july01.csv')

(176217, 2)


In [7]:
# # for selection of images for binary labeling - halo_failure_case_of_box_in_dust
# selected = [3, 5, [9, 1/5], [10, 1/5], [11, 1/6], 
#     [12, [['2024-04-23T18:11:28', '2024-04-23T18:18:12'], ['2024-04-23T18:27:40', '2024-04-23T18:28:38'], ['2024-04-23T18:29:05', '2024-04-23T18:29:57'], 
#         ['2024-04-23T18:48:44', '2024-04-23T18:50:59'], ['2024-04-23T19:01:50', '2024-04-23T19:03:37']]], 
#     [13, [['2024-04-23T19:47:48', '2024-04-23T19:47:59']]], 
#     [14, [['2024-04-23T20:12:40', '2024-04-23T20:13:02'], ['2024-04-23T20:15:53', '2024-04-23T20:17:49'], ['2024-04-23T20:22:19', '2024-04-23T20:22:45'], 
#         ['2024-04-23T20:32:47', '2024-04-23T20:33:18'], ['2024-04-23T20:34:48', '2024-04-23T20:35:56'], ['2024-04-23T20:38:00', '2024-04-23T20:38:17'], 
#         ['2024-04-23T20:39:55', '2024-04-23T20:41:31'], ['2024-04-23T20:42:57', '2024-04-23T20:48:27']]], 
#     [16, 1/2]]
# selected_ids = []
# for s in selected:
#     if isinstance(s, int):
#         selected_ids += seq_dfs[s].id.to_list()
#     else:
#         si, sj = s
#         if isinstance(sj, float):
#             selected_ids += seq_dfs[si].iloc[:int(len(seq_dfs[si])*sj)].id.to_list()
#         else:
#             for t1, t2 in sj:
#                 selected_ids += seq_dfs[si][(seq_dfs[si].collected_on >= t1) & (seq_dfs[si].collected_on <= t2)].id.to_list()
#     # print(s, len(selected_ids))
# selected_ids = list(set(selected_ids))
# selected_df = pd.DataFrame(data={'id': selected_ids})
# print(selected_df.shape)
# selected_df.to_csv('/data/jupiter/datasets/halo_failure_case_of_box_in_dust/selected_for_label.csv', index=False)

In [7]:
def select_ids_for_label(seq_dfs, all_cameras, selected, save_csv):
    selected_ids = []
    for pod, cameras in all_cameras.items():
        print(pod, cameras)
        for i,seq_df in enumerate(seq_dfs):
            if i in selected[pod]:
                sub_df = seq_df[seq_df.camera_location.isin(cameras)]
                selected_ids += sub_df.id.to_list()
    selected_ids = list(set(selected_ids))
    selected_df = pd.DataFrame(data={'id': selected_ids})
    print(selected_df.shape)
    if save_csv:
        selected_df.to_csv(save_csv, index=False)
        return None
    return selected_df

def recover_skipped_human_label(data_dir, seq_dfs, suffix, same_human_sequence):
    all_cameras = {'front': ['T01', 'T02', 'T03', 'T04'], 'right': ['T05', 'T06', 'T07', 'T08'], 'back': ['T09', 'T10', 'T11', 'T12'], 'left': ['T13', 'T14', 'T15', 'T16']}

    raw_ms_df = pd.read_csv(os.path.join(data_dir, 'master_annotations.csv'))
    raw_ms_df['camera_pair'] = raw_ms_df['unique_id'].apply(lambda s: s[-7:])
    raw_ms_df['rectified_label_save_path'] = ''
    raw_ms_df['use_recovered_label_in_sequence'] = True
    labeled_ms_df = pd.read_csv(os.path.join(data_dir+suffix, 'master_annotations.csv'))
    labeled_ms_df.drop(columns=["label_counts"], inplace=True)
    labeled_ms_df['camera_pair'] = labeled_ms_df['unique_id'].apply(lambda s: s[-7:])
    print(raw_ms_df.shape, labeled_ms_df.shape)

    recovered_dfs = []
    for pod, seq_ids in same_human_sequence.items():
        for seq_id in seq_ids:
            # get seq df in pod
            seq_df = seq_dfs[seq_id]
            seq_df = seq_df[seq_df.camera_location.isin(all_cameras[pod])]
            # get corresponding raw seq df and labeled seq df
            raw_seq_df = raw_ms_df[raw_ms_df.id.isin(seq_df.id)]
            labeled_seq_df = labeled_ms_df[labeled_ms_df.id.isin(seq_df.id)]
            labeled_seq_df = labeled_seq_df.sort_values('collected_on')
            # get camera locations where there are human labels
            labeled_camera_pairs = labeled_seq_df.camera_pair.unique()
            for camera_pair in labeled_camera_pairs:
                raw_seq_cp_df = raw_seq_df[raw_seq_df.camera_pair == camera_pair]
                labeled_seq_cp_df = labeled_seq_df[labeled_seq_df.camera_pair == camera_pair]
                # assign label path to raw df
                for i, row in raw_seq_cp_df.iterrows():
                    labeled_rows = labeled_seq_cp_df[labeled_seq_cp_df.unique_id == row.unique_id]
                    if len(labeled_rows) > 0:
                        raw_seq_cp_df.loc[i, 'rectified_label_save_path'] = labeled_rows.iloc[0].rectified_label_save_path
                        raw_seq_cp_df.loc[i, 'use_recovered_label_in_sequence'] = False
                    else:
                        raw_seq_cp_df.loc[i, 'rectified_label_save_path'] = labeled_seq_cp_df.iloc[0].rectified_label_save_path
                recovered_dfs.append(raw_seq_cp_df)
            print(pod, seq_id, len(seq_df), len(raw_seq_df), len(labeled_seq_df), raw_seq_df.camera_pair.unique(), labeled_seq_df.camera_pair.unique())
    recovered_df = pd.concat(recovered_dfs, ignore_index=True)
    print(recovered_df.shape, labeled_ms_df.shape)

    # remove duplicates and add in extra sequence images from labeled_ms_df
    recovered_df = pd.concat([recovered_df.drop_duplicates(subset='unique_id'), labeled_ms_df[~labeled_ms_df.unique_id.isin(recovered_df.unique_id)]], ignore_index=True)
    # change relative path to label path
    recovered_df['rectified_label_save_path'] = recovered_df['rectified_label_save_path'].apply(lambda p: f'../{os.path.basename(data_dir)+suffix}/{p}')
    recovered_df['label_map'] = labeled_ms_df.iloc[0].label_map
    print(recovered_df.shape, labeled_ms_df.shape)
    recovered_df.to_csv(os.path.join(data_dir, 'label_recovered_master_annotations.csv'), index=False)

In [8]:
# # for selection of images for partial human labeling - halo_human_in_dust_day_collection_may29
# selected = {'front': {0: 'all', 1: ['begin', '2024-05-29T19:48:13'], 2: ['2024-05-29T20:28:50', 'end']},
#             'right': {2: ['begin', '2024-05-29T20:28:53']},
#             'back': {1: ['2024-05-29T20:02:08', 'end']},
#             'left': {1: ['2024-05-29T19:48:10', '2024-05-29T20:02:08']}}
# all_cameras = {'front': ['T01', 'T02', 'T03', 'T04'], 'right': ['T05', 'T06', 'T07', 'T08'], 'back': ['T09', 'T10', 'T11', 'T12'], 'left': ['T13', 'T14', 'T15', 'T16']}
# left_cameras = ['front-center-left', 'front-left-left', 'front-right-left', 'side-left-left', 'side-right-left', 'rear-left', 'T01', 'T02', 'T05', 'T06', 'T09', 'T10', 'T13', 'T14', 'I01', 'I02']

# selected_ids = []
# for pod, cameras in all_cameras.items():
#     cameras = set(cameras).intersection(left_cameras)
#     selected_times = selected[pod]
#     for seq_i, t1t2 in selected_times.items():
#         seq_df = seq_dfs[seq_i]
#         seq_df = seq_df[seq_df.camera_location.isin(cameras)]
#         if t1t2 == 'all':
#             t1, t2 = seq_df.iloc[0].collected_on, seq_df.iloc[-1].collected_on
#         else:
#             t1, t2 = t1t2
#             if t1 == 'begin':
#                 t1 = seq_df.iloc[0].collected_on
#             if t2 == 'end':
#                 t2 = seq_df.iloc[-1].collected_on
#         print(pod, seq_i, t1, t2, len(seq_df))
#         selected_ids += seq_df[(seq_df.collected_on >= t1) & (seq_df.collected_on <= t2)].id.to_list()
# selected_ids = list(set(selected_ids))
# selected_df = pd.DataFrame(data={'id': selected_ids})
# print(selected_df.shape)
# selected_df.to_csv('/data/jupiter/datasets/halo_human_in_dust_day_collection_may29/selected_for_label.csv', index=False)

# # recover skipped human labels in heavy dust, by images in the same sequence
# same_human_sequence = {'front': [0, 1, 3, 14], 'right': [11, 12, 13], 'back': [7, 8, 9, 10], 'left': [4, 5, 6]}
# suffix = '_human_labeled_stereo'
# recover_skipped_human_label(data_dir, seq_dfs, suffix, same_human_sequence)

# select images with dust on lens
left_cameras = {'front': ['T01', 'T02'], 'right': ['T05', 'T06'], 'back': ['T09', 'T10'], 'left': ['T13', 'T14']}
# selected = {'front': [1, 3, 5, 11, 14, 15], 'right': [15,], 'back': [], 'left': [6, 10]}  # all
# selected = {'front': [1, 3, 14], 'right': [], 'back': [], 'left': [6]}  # with human
selected = {'front': [5, 11], 'right': [15,], 'back': [], 'left': [10]}  # without human
save_csv = '/data/jupiter/datasets/halo_human_in_dust_day_collection_may29/dust_on_lens_without_human.csv'
select_ids_for_label(seq_dfs, left_cameras, selected, save_csv)

front ['T01', 'T02']
right ['T05', 'T06']
back ['T09', 'T10']
left ['T13', 'T14']
(1048, 1)


In [18]:
# # load in labeled dataset
# labeled_csv = '/data2/jupiter/datasets/halo_vehicles_driving_through_dust_images_nodust_reserved_labeled_maxfov_alleysson_depth0423/annotations.csv'
# labeled_df = pd.read_csv(labeled_csv)
# print(labeled_df.shape)
# labeled_ids = set(labeled_df.id.to_list())
# print(len(labeled_ids))

# # for selection of images for missing partial vehicle labeling - halo_vehicles_in_dust_collection_march2024
# left_cameras = {'left_pass': ['T09', 'T14', 'T13', 'T02'], 'right_pass': ['T10', 'T05', 'T06', 'T01']}
# selected = {'left_pass': [2, 3, 4, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 27, 28, 30, 31, 32, 33, 34, 35, 36], 
#             'right_pass': [3, 4, 5, 6, 7, 9, 10, 12, 13, 14, 15, 16, 17, 18, 20, 21, 26, 27, 37]}
# selected_df = select_ids_for_label(seq_dfs, left_cameras, selected, save_csv='')
# print(selected_df.shape)
# selected_df = selected_df[~selected_df.id.isin(labeled_ids)]
# print(selected_df.shape)
# selected_df.to_csv('/data/jupiter/datasets/halo_vehicles_in_dust_collection_march2024/selected_for_missing_label.csv', index=False)

left_pass ['T09', 'T14', 'T13', 'T02']
right_pass ['T10', 'T05', 'T06', 'T01']
(13141, 1)
(9893, 1)


In [11]:
# # for selection of images for missing partial human labeling - halo_human_in_dust_night_collection_june03
# left_cameras = {'front': ['T01', 'T02'], 'right': ['T05', 'T06'], 'back': ['T09', 'T10'], 'left': ['T13', 'T14']}
# all_cameras = {'front': ['T01', 'T02', 'T03', 'T04'], 'right': ['T05', 'T06', 'T07', 'T08'], 'back': ['T09', 'T10', 'T11', 'T12'], 'left': ['T13', 'T14', 'T15', 'T16']}
# selected = {'front': [25, 26, 27, 30]}
# select_ids_for_label(seq_dfs, left_cameras, selected, save_csv='')

# # for selection of images for missing partial human labeling - halo_human_in_dust_night_collection_june03_2
# left_cameras = {'front': ['T01', 'T02'], 'right': ['T05', 'T06'], 'back': ['T09', 'T10'], 'left': ['T13', 'T14']}
# selected = {'front': [0, 1, 2, 3, 15, 16, 17], 'right': [18,], 'back': [7, 8, 9, 10, 11], 'left': [4, 5, 6, 12, 13, 14]}
# save_csv = '/data/jupiter/datasets/halo_human_in_dust_night_collection_june03_2/selected_for_label.csv'
# select_ids_for_label(seq_dfs, left_cameras, selected, save_csv)

# # recover skipped human labels in heavy dust, by images in the same sequence
# same_human_sequence = {'front': [15, 17], 'right': [18], 'back': [7, 8, 9, 10], 'left': [5, 6]}
# suffix = '_human_labeled_stereo'
# recover_skipped_human_label(data_dir, seq_dfs, suffix, same_human_sequence)

# select images with dust on lens
left_cameras = {'front': ['T01', 'T02'], 'right': ['T05', 'T06'], 'back': ['T09', 'T10'], 'left': ['T13', 'T14']}
# selected = {'front': [], 'right': [], 'back': [], 'left': [12, 13, 14]}  # with human
selected = {'front': [16], 'right': [], 'back': [], 'left': []}  # without human
save_csv = '/data/jupiter/datasets/halo_human_in_dust_night_collection_june03_2/dust_on_lens_without_human.csv'
select_ids_for_label(seq_dfs, left_cameras, selected, save_csv)

front ['T01', 'T02']
right ['T05', 'T06']
back ['T09', 'T10']
left ['T13', 'T14']
(674, 1)


In [64]:
# for selection of images for missing partial human labeling - halo_human_in_dust_day_collection_back_june05
# recover skipped human labels in heavy dust, by images in the same sequence
same_human_sequence = {'back': [0, 1, 2, 3, 5, 6, 7]}
suffix = '_human_labeled_stereo'
recover_skipped_human_label(data_dir, seq_dfs, suffix, same_human_sequence)

<ipython-input-64-9e5b735fb503>:5: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  recovered_df, labeled_ms_df = recover_skipped_human_label(data_dir, seq_dfs, suffix, same_human_sequence)
/home/li.yu/anaconda3/envs/pytorchlightning/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


(8740, 153) (3511, 277)
back 0 752 562 184 ['T09_T11' 'T10_T12' 'T10_T11'] ['T09_T11']
back 1 1216 911 114 ['T09_T11' 'T10_T12' 'T10_T11'] ['T09_T11']
back 2 248 186 11 ['T09_T11' 'T10_T12' 'T10_T11'] ['T09_T11']
back 3 1476 1102 196 ['T09_T11' 'T10_T12' 'T10_T11'] ['T09_T11']
back 5 1688 1262 576 ['T09_T11' 'T10_T12' 'T10_T11'] ['T10_T12' 'T10_T11']
back 6 2176 1631 967 ['T09_T11' 'T10_T12' 'T10_T11'] ['T10_T11' 'T10_T12']
back 7 1380 1034 687 ['T09_T11' 'T10_T12' 'T10_T11'] ['T10_T12' 'T10_T11']
(3538, 153) (3511, 277)
(4314, 278) (3511, 277)


In [8]:
# # for selection of images for missing partial human labeling - halo_human_in_dust_dusk_collection_front_june07
# left_cameras = {'front': ['T01', 'T02'], 'right': ['T05', 'T06'], 'back': ['T09', 'T10'], 'left': ['T13', 'T14']}
# selected = {'front': [5], 'right': [], 'back': [], 'left': []}
# save_csv = '/data/jupiter/datasets/halo_human_in_dust_dusk_collection_front_june07/selected_for_label.csv'
# select_ids_for_label(seq_dfs, left_cameras, selected, save_csv)

front ['T01', 'T02']
right ['T05', 'T06']
back ['T09', 'T10']
left ['T13', 'T14']
(3354, 1)


In [14]:
# # for selection of images for missing partial vehicle labeling - halo_vehicles_in_dust_collection_june06
# left_cameras = {'left_pass': ['T09', 'T14', 'T13', 'T02'], 'right_pass': ['T10', 'T05', 'T06', 'T01']}
# selected = {'left_pass': [11, 12, 15], 
#             'right_pass': [10, 13, 14]}
# save_csv = '/data/jupiter/datasets/halo_vehicles_in_dust_collection_june06/selected_for_label.csv'
# select_ids_for_label(seq_dfs, left_cameras, selected, save_csv)

left_pass ['T09', 'T14', 'T13', 'T02']
right_pass ['T10', 'T05', 'T06', 'T01']
(4513, 1)


In [8]:
# # for selection of images for missing partial vehicle labeling - halo_vehicles_in_dust_collection_june07
# left_cameras = {'left_pass': ['T09', 'T14', 'T13', 'T02'], 'right_pass': ['T10', 'T05', 'T06', 'T01']}
# selected = {'left_pass': [2, 6, 7, 10], 
#             'right_pass': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]}
# save_csv = '/data/jupiter/datasets/halo_vehicles_in_dust_collection_june07/selected_for_label.csv'
# select_ids_for_label(seq_dfs, left_cameras, selected, save_csv)

left_pass ['T09', 'T14', 'T13', 'T02']
right_pass ['T10', 'T05', 'T06', 'T01']
(14354, 1)


In [9]:
# # for selection of images for missing partial vehicle labeling - halo_vehicles_in_dust_collection_june04
# left_cameras = {'left_pass': ['T09', 'T14', 'T13', 'T02'], 'right_pass': ['T10', 'T05', 'T06', 'T01']}
# selected = {'left_pass': [1, 2, 3, 4, 5, 6, 7, 8, 9], 
#             'right_pass': [9]}
# save_csv = '/data/jupiter/datasets/halo_vehicles_in_dust_collection_june04/selected_for_label.csv'
# select_ids_for_label(seq_dfs, left_cameras, selected, save_csv)

left_pass ['T09', 'T14', 'T13', 'T02']
right_pass ['T10', 'T05', 'T06', 'T01']
(26470, 1)


In [9]:
def select_ids_for_label_v2(seq_dfs, selected, save_csv):
    selected_ids = []
    for i,seq_df in enumerate(seq_dfs):
        for camera, slices in selected.items():
            cam_df = seq_df[seq_df.camera_location == camera]
            for slice in slices:
                if isinstance(slice, list):
                    if slice[0] != i:
                        continue
                    cam_df = cam_df.sort_values('collected_on')
                    begin = end = ''
                    if slice[1] == 'begin':
                        begin = cam_df.iloc[0].collected_on
                    elif slice[1] == 'middle':
                        begin = cam_df.iloc[len(cam_df)//2].collected_on
                    else:
                        begin = slice[1]
                    if slice[2] == 'middle':
                        end = cam_df.iloc[len(cam_df)//2].collected_on
                    elif slice[2] == 'end':
                        end = cam_df.iloc[-1].collected_on
                    else:
                        end = slice[2]
                    sub_df = cam_df[(cam_df.collected_on >= begin) & (cam_df.collected_on <= end)]
                    selected_ids += sub_df.id.to_list()
                    print(camera, i, len(sub_df))
                else:
                    if slice != i:
                        continue
                    selected_ids += cam_df.id.to_list()
                    print(camera, i, len(cam_df))
    selected_ids = list(set(selected_ids))
    selected_df = pd.DataFrame(data={'id': selected_ids})
    print(selected_df.shape)
    if save_csv:
        selected_df.to_csv(save_csv, index=False)
        return None
    return selected_df

In [14]:
# # for selection of images for running PP and model inference - Jupiter_bedrock_40013_20240617_dust_sequences
# all_cameras = {'front': ['T01', 'T02', 'T03', 'T04'], 'right': ['T05', 'T06', 'T07', 'T08'], 'back': ['T09', 'T10', 'T11', 'T12'], 'left': ['T13', 'T14', 'T15', 'T16']}
# selected = {'front': [0, 1, 4, 8, 9], 'right': [], 'back': [], 'left': []}
# save_csv = '/data/jupiter/datasets/dust_datasets/Jupiter_bedrock_40013_20240617_dust_sequences/selected_for_pp.csv'
# select_ids_for_label(seq_dfs, all_cameras, selected, save_csv)
# selected_ids_df = pd.read_csv(save_csv)
# selected_ann_df = df[df.id.isin(selected_ids_df.id)]
# print(selected_ann_df.shape)
# selected_ann_df.to_csv('/data/jupiter/datasets/dust_datasets/Jupiter_bedrock_40013_20240617_dust_sequences/selected_for.csv', index=False)

# select dust on lens only (no dust in air) sequences:
selected_box = {'T02': [[1, '2024-06-17T21:24:51.901000', 'end'], 2, 3]}
selected_human = {'T01': [[10, 'begin', '2024-06-17T21:53:49:495000']], 
            'T02': [[4, 'begin', '2024-06-17T21:32:18.787000'], [9, 'middle', 'end'], [10, 'begin', '2024-06-17T21:53:49:495000']]}
selected_human_v2 = {'T02': [[4, '2024-06-17T21:28:44.335000', '2024-06-17T21:32:18.787000'], [9, '2024-06-17T21:45:47.644000', 'end']]}
selected_box_csv = os.path.join(data_dir, 'dust_on_lens_with_box.csv')
selected_human_csv = os.path.join(data_dir, 'dust_on_lens_with_human.csv')
selected_human_v2_csv = os.path.join(data_dir, 'dust_on_lens_with_human_v3.csv')
# select_ids_for_label_v2(seq_dfs, selected_box, selected_box_csv)
# select_ids_for_label_v2(seq_dfs, selected_human, selected_human_csv)
select_ids_for_label_v2(seq_dfs, selected_human_v2, selected_human_v2_csv)

# # select dust on lens and dust in air with manny sequences
# selected_human = {'T02': [[9, 'begin', 'middle']]}
# selected_human_csv = os.path.join(data_dir, 'mixed_dust_with_human.csv')
# select_ids_for_label_v2(seq_dfs, selected_human, selected_human_csv)

T02 4 645
T02 9 3
(648, 1)


In [31]:
# for selection and triaging of dust on lens IQ test set - halo_dust_on_lens_blur_dataset_v3_20240807
all_cameras = {'front': ['T01', 'T02', 'T03', 'T04'], 'right': ['T05', 'T06', 'T07', 'T08'], 'back': ['T09', 'T10', 'T11', 'T12'], 'left': ['T13', 'T14', 'T15', 'T16']}
dedup = {'front': {0:[1, 3], 1: [1, 3], 8: [1, 3, 4], 9: [1,2,3,4], 10: [1,2,3,4], 17:[1, 3], 18:[1, 3], 19:[1, 3], 20:[1, 3], 21:[1, 3], 22:[1, 3], 23:[1, 3], 
                   24:[1, 3], 25:[1, 3], 26:[1, 3], 32:[1, 3], 33:[1, 3], 36:[1,2,3,4], 37:[1,2,3,4], 38:[1,2,3,4], 39:[1,2,3,4], 40:[1,2,3,4], 41:[1,2], 43:[1,2]}, 
         'right': {2:[5,7], 3:[5,7], 4:[5,7], 11:[5,6,7,8], 12:[5,6,7,8], 28:[7], 29:[7], 36:[5,6,7,8], 37:[5,6,7,8], 38:[5,6,7,8], 39:[5,6,7,8], 40:[5,6,7,8]}, 
         'back': {13:[9,10,11,12], 14:[9,10,11,12], 36:[9,10,11,12], 37:[9,10,11,12], 38:[9,10,11,12], 39:[9,10,11,12], 40:[9,10,11,12]}, 
         'left': {5:[13,15], 8:[16], 9:[16], 10:[16], 11:[16], 12:[16], 13:[16], 14:[16], 15:[13,14,15,16], 16:[13,14,15,16], 27:[14,16], 36:[13,14,15,16], 37:[13,14,15,16], 
                  38:[13,14,15,16], 39:[13,14,15,16], 40:[13,14,15,16], 42:[13,14,15,16]}}
remove = {'front': [4, 6, 14, 15, 45], 'right': [6, 13, 14, 15, 16, 34, 44], 'back': [8, 11, 15, 16, 26, 27, 30, 31, 32, 33, 34, 35], 'left': [6, 7, 24, 28, 31]}
# load already triaged ids
triage_df = pd.read_csv(os.path.join(data_dir, 'to_triage.csv'))
print(triage_df.shape)
# dedup images from each sequence, such that there is at most 30 images per minute per camera
pods = ['front', 'left', 'right', 'back']
limit = 30
selected_dfs = []
for i, seq_df in enumerate(seq_dfs):
    for pod in pods:
        # check for remove list
        if i in remove[pod]:
            continue
        # dedup
        if i in dedup[pod]:
            cameras = [f'T{str(seq_i).zfill(2)}' for seq_i in dedup[pod][i]]
            for cam in cameras:
                cam_df = seq_df[seq_df.camera_location == cam]
                cam_df = cam_df[~cam_df.id.isin(triage_df.id)]
                if len(cam_df) == 0:
                    continue
                cam_df = cam_df.sort_values('collected_on')
                time_span = (cam_df.iloc[-1].datetime - cam_df.iloc[0].datetime).total_seconds()
                limit_ids = int(time_span / 60 * limit)
                selected_dfs.append(cam_df.sample(min(len(cam_df), limit_ids)))
                # print(i, pod, cam, time_span, len(cam_df), limit_ids)
        # else:
        #     pod_df = seq_df[seq_df.camera_location.isin(all_cameras[pod])]
        #     selected_dfs.append(pod_df)
        #     print(i, pod, cam, len(pod_df))
selected_df = pd.concat(selected_dfs, ignore_index=True)
print(df.shape, selected_df.shape)
# save ids to csv
selected_df[['id']].to_csv(os.path.join(data_dir, 'v4_dedup_triaged.csv'), index=False)

(1085, 1)
(19813, 149) (8534, 150)


### create video

In [5]:
def read_csvs(pred_root_dir, model, labeled_dataset):
    pred_df = pd.read_csv(os.path.join(pred_root_dir, model, labeled_dataset, 'output.csv'))
    if not 'state' in pred_df:
        pred_df['state'] = pred_df['result_state']
    dust_df = pd.read_csv(os.path.join(pred_root_dir, model, labeled_dataset, 'dust_ratio.csv'))
    print(labeled_dataset, pred_df.shape, dust_df.shape)
    df = pred_df[['unique_id', 'state', 'camera_location', 'operation_time']].merge(dust_df, on='unique_id')
    all_cameras = {'Front Pod': ['T01', 'T02', 'T03', 'T04'], 'Right Pod': ['T05', 'T06', 'T07', 'T08'], 'Rear Pod': ['T09', 'T10', 'T11', 'T12'], 'Left Pod': ['T13', 'T14', 'T15', 'T16']}
    all_cameras = {c: pod for pod, cameras in all_cameras.items() for c in cameras}
    df['pod'] = df.camera_location.apply(lambda c: all_cameras[c])
    return df

In [18]:
# load in model predictions
pred_root_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model/'
model = '9_2_unofficial_rev1_lying_down_sitting_25_ep_prod_weights_10_lo_10_tr_br_dr_p2_v_p1_m_p1_u_dust_0624'
# model = '9_2_unofficial_rev1_lying_down_sitting_25_ep_prod_weights_10_lo_5_tr_br_dr_p2_v_p1_m_p05_u_2_dust_0626'
# unlabeled pred
pred_df = read_csvs(pred_root_dir, model, os.path.basename(data_dir))
print(pred_df.shape)
# # labeled pred
# suffix = '_human_labeled_stereo'
# labeled_pred_df = read_csvs(pred_root_dir, model, os.path.basename(data_dir)+suffix)
# print(labeled_pred_df.shape)
# # put labeled rows to raw, unlabeled df
# pred_df.loc[pred_df.unique_id.isin(labeled_pred_df.unique_id), ['total_averaged_dust_conf', 'state']] = labeled_pred_df[['total_averaged_dust_conf', 'state']].values
pred_df.head(2)

Jupiter_bedrock_40013_20240617_dust_sequences (6207, 24) (1552, 7)
(1552, 11)


,unique_id,state,camera_location,operation_time,id,gt_dust_ratio,total_averaged_dust_conf,total_thresholded_dust_ratio,masked_avg_dust_conf,masked_dust_ratio,pod
0,667cb90b71111a991df8b4a3_T02_T03,true_negative,T02,daytime,667cb90b71111a991df8b4a3,0.0,0.000408,0.0,0.000002,0.0,Front Pod
1,667cb6533ba692c3dd8db0b4_T02_T03,true_negative,T02,daytime,667cb6533ba692c3dd8db0b4,0.0,0.000568,0.0,0.000002,0.0,Front Pod


In [20]:
seq_df = seq_dfs[6]
pred_seq_df = pred_df[pred_df.id.isin(seq_df.id)]
print(seq_df.shape, pred_seq_df.shape)
pred_seq_df.groupby('state').size()

(4308, 109) (3218, 11)


state
false_negative       1
false_positive    1171
true_negative     1672
true_positive      374
dtype: int64

In [6]:
# read from multiple cameras and put in once frame
# left_pass_pairs = ['T09_T11', 'T14_T16', 'T14_T15', 'T13_T15']
# right_pass_pairs = ['T05_T07', 'T10_T12', 'T06_T08', 'T06_T07']
# cameras = [f'T{str(i+1).zfill(2)}' for i in range(16)][12:]
# cameras = ['T16', 'T15', 'T14', 'T13', 'T11', 'T09']
# all_cameras = {'front': ['T01', 'T02', 'T03', 'T04'], 'right': ['T05', 'T06', 'T07', 'T08'], 'back': ['T09', 'T10', 'T11', 'T12'], 'left': ['T13', 'T14', 'T15', 'T16']}
all_left_cameras = {'front': ['T01', 'T02'], 'right': ['T05', 'T06'], 'back': ['T09', 'T10'], 'left': ['T13', 'T14']}
# left_cameras = ['T01', 'T02', 'T05', 'T06', 'T09', 'T10', 'T13', 'T14']
# back_left_cameras = ['T09', 'T10']
# left_cameras = {'left_pass': ['T09', 'T14', 'T13', 'T02'], 'right_pass': ['T10', 'T05', 'T06', 'T01']}
# selected = {'front': [1, 3, 5, 11, 14, 15], 'right': [15,], 'back': [], 'left': [6, 10]}
for pod, cameras in all_left_cameras.items():
    # pod = 'all_left'
    # cameras = left_cameras
    print(pod, cameras)
    H = 1  # number of camera rows
    W = 2  # number of camera cols
    for i,seq_df in enumerate(seq_dfs):
        # # select sequence to create video
        # if not i in selected[pod]:
        #     continue
        # # check model prediction and filter by dust/seg outputs
        # pred_seq_df = pred_df[pred_df.id.isin(seq_df.id)]
        # labeled_ids = pred_seq_df.id.to_list()

        cam_dfs = [seq_df[seq_df.camera_location == c] for c in cameras]
        print(i, [len(cdf) for cdf in cam_dfs])
        min_len = min(len(cdf) for cdf in cam_dfs)
        cam_dfs = [cdf.sort_values('collected_on').iloc[:min_len] for cdf in cam_dfs]
        if min_len < 2:
            continue
        # print(i, [cdf.iloc[0].collected_on for cdf in cam_dfs])

        frame = read_raw_image_by_row(data_dir, seq_df.iloc[0])
        height, width, layers = frame.shape
        # print(height, width, layers)

        # .avi MJPG,  .mp4 MP4V
        video_dir = os.path.join(data_dir, f'videos_{len(cameras)}cams')
        os.makedirs(video_dir, exist_ok=True)
        video_name = os.path.join(video_dir, f'{pod}_seq{str(i).zfill(2)}.mp4')
        video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), 3, (width*W,height*H), isColor=True)

        for ii in tqdm(range(min_len)):
            try:
                canvas = np.zeros((height*H, width*W, 3), dtype=np.uint8)
                for ci in range(len(cam_dfs)):
                    cam_df_row = cam_dfs[ci].iloc[ii]
                    frame = read_raw_image_by_row(data_dir, cam_df_row)
                    c = (255,0,0)
                    s = f'{cam_df_row.camera_location} {cam_df_row.collected_on}'
                    # if cam_df_row.id in labeled_ids:
                    #     pred_row = pred_seq_df[pred_seq_df.id == cam_df_row.id].iloc[0]
                    #     s += f' pred: {pred_row.state}, dust: {pred_row.total_averaged_dust_conf:.4f}'
                    #     if pred_row.state == 'true_positive' or pred_row.state == 'false_positive':
                    #         c = (0,0,255)
                    frame = cv2.putText(frame, s, 
                            (40,50), cv2.FONT_HERSHEY_SIMPLEX, 1, c, 2, cv2.LINE_AA)
                    fi, fj = ci // W, ci % W
                    canvas[height*fi:height*(fi+1), width*fj:width*(fj+1)] = frame
                video.write(canvas)
            except:
                print(f'{ii}th image read failed')

        # cv2.destroyAllWindows()
        video.release()
        # break

  0%|          | 0/128 [00:00<?, ?it/s]

front ['T01', 'T02']
0 [128, 128]


  1%|          | 1/100 [00:00<00:12,  8.10it/s]

1 [100, 100]


  1%|          | 1/112 [00:00<00:14,  7.84it/s]

...

9 [212, 211]


  0%|          | 1/614 [00:00<01:16,  8.07it/s]

10 [614, 614]


100%|██████████| 614/614 [01:20<00:00,  7.61it/s]

11 [0, 0]
12 [0, 0]
13 [0, 0]
14 [0, 0]


In [5]:
root_dir = '/data/jupiter/datasets/'
dataset = 'halo_dust_on_lens_blur_dataset_v3_20240807'
sample_csv = os.path.join(root_dir, dataset, 'iq_fn_depth_smudge_halo_dust_on_lens_blur_dataset_v3_20240807.csv')
sdf = pd.read_csv(sample_csv)
sdf['id'] = sdf['unique_id'].apply(lambda s: s[:-8])
df = df[df.id.isin(sdf.id)]
print(f'{df.shape}')

(2652, 149)


In [7]:
save_dir = os.path.join(root_dir, dataset, 'fns')
os.makedirs(save_dir, exist_ok=True)

# all_cameras = {'front': ['T01', 'T02', 'T03', 'T04'], 'right': ['T05', 'T06', 'T07', 'T08'], 'back': ['T09', 'T10', 'T11', 'T12'], 'left': ['T13', 'T14', 'T15', 'T16']}
all_cameras = {'back': ['T09', 'T10', 'T11', 'T12']}
for pod, cameras in all_cameras.items():
    print(pod, cameras)
    for i,seq_df in enumerate(seq_dfs):
        seq_df_pod = seq_df[(seq_df.id.isin(sdf.id)) & (seq_df.camera_location.isin(cameras))]
        cam_dfs = [seq_df_pod[seq_df_pod.camera_location == c] for c in cameras]
        print(i, [len(cdf) for cdf in cam_dfs])
        max_len = max(len(cdf) for cdf in cam_dfs)
        if max_len == 0:
            continue
        
        save_dir_pod_seq = os.path.join(save_dir, f'{pod}_{i}')
        os.makedirs(save_dir_pod_seq, exist_ok=True)
        for _, row in seq_df_pod.iterrows():
            img_path = os.path.join(root_dir, dataset, row.artifact_debayeredrgb_0_save_path)
            img = cv2.imread(img_path)
            cv2.imwrite(os.path.join(save_dir_pod_seq, f'{row.id}.jpg'), img)

back ['T09', 'T10', 'T11', 'T12']
0 [0, 0, 0, 0]
1 [0, 0, 0, 0]
2 [0, 0, 0, 0]
3 [0, 0, 0, 0]
4 [0, 0, 0, 0]
5 [0, 0, 0, 0]
6 [0, 0, 0, 0]
7 [0, 0, 0, 0]
8 [1, 2, 0, 0]
9 [0, 0, 0, 0]
10 [0, 0, 0, 0]
11 [6, 6, 0, 0]
12 [0, 0, 0, 0]
13 [0, 0, 0, 0]
14 [0, 3, 0, 0]
15 [2, 7, 0, 0]
16 [0, 3, 0, 0]
17 [0, 0, 0, 0]
18 [0, 0, 0, 0]
19 [0, 0, 0, 0]
20 [0, 0, 0, 0]
21 [0, 0, 0, 0]
22 [0, 0, 0, 0]
23 [0, 0, 0, 0]
24 [0, 0, 0, 0]
25 [0, 0, 0, 0]
26 [0, 0, 0, 0]
27 [0, 1, 0, 0]
28 [0, 0, 0, 0]
29 [0, 0, 0, 0]
30 [0, 0, 0, 0]
31 [0, 2, 0, 0]
32 [0, 0, 0, 0]
33 [0, 0, 0, 0]
34 [0, 0, 0, 0]
35 [0, 1, 0, 0]
36 [78, 73, 0, 0]
37 [94, 82, 0, 0]
38 [0, 0, 0, 0]
39 [6, 59, 0, 0]
40 [36, 73, 0, 0]
41 [0, 0, 0, 0]
42 [0, 0, 0, 0]
43 [0, 0, 0, 0]
44 [0, 0, 0, 0]
45 [0, 0, 0, 0]


## Create video from saved pngs

In [56]:
def create_video(ids, pred_dir, video_name, read_func=read_saved_frame, fps=2, txt_df=None):
    frame = read_func(pred_dir, ids[0])
    height, width, layers = frame.shape
    print(height, width, layers)

    # .avi MJPG,  .mp4 MP4V
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), fps, (width,height), isColor=True)
    
    good = 0
    for _id in tqdm(ids):
        frame = read_func(pred_dir, _id)
        if frame is not None:
            if txt_df is not None:
                txt_row = txt_df[txt_df.unique_id == _id].iloc[0]
                c = (255,0,0)
                s = f'{txt_row.collected_on} dust ratio: {txt_row.total_averaged_dust_conf:.4f}'
                frame = cv2.putText(frame, s, 
                        (600,60), cv2.FONT_HERSHEY_SIMPLEX, 1, c, 2, cv2.LINE_AA)
            video.write(frame)
            good += 1
    print('total', len(ids), 'used', good)

    # cv2.destroyAllWindows()
    video.release()

In [58]:
dust_df.total_averaged_dust_conf.describe()

count    537.000000
mean       0.000291
std        0.000210
min        0.000004
25%        0.000129
50%        0.000247
75%        0.000431
max        0.001360
Name: total_averaged_dust_conf, dtype: float64

In [71]:
dust_df[dust_df.collected_on == '2024-06-17T21:44:49.036000']

,unique_id,collected_on,id,gt_dust_ratio,total_averaged_dust_conf,total_thresholded_dust_ratio,masked_avg_dust_conf,masked_dust_ratio
261,667c621f79de73c09b0e71ab_T02_T04,2024-06-17T21:44:49.036000,667c621f79de73c09b0e71ab,0.0,0.000144,0.0,2.366287e-06,0.0
262,667c621b1280b4ae671bcdcc_T01_T03,2024-06-17T21:44:49.036000,667c621b1280b4ae671bcdcc,0.0,0.000279,0.0,2.814136e-07,0.0
263,667c621f79de73c09b0e71ab_T02_T03,2024-06-17T21:44:49.036000,667c621f79de73c09b0e71ab,0.0,0.000864,0.0,1.475281e-07,0.0


In [63]:
master_df.camera_location.unique(), master_df.camera_location_right.unique()

(array(['T02', 'T01'], dtype=object), array(['T03', 'T04'], dtype=object))

In [70]:
data_dir = '/data/jupiter/datasets/dust_datasets/'
# dataset = 'Jupiter_bedrock_40013_20240617_dust_sequences'
dataset = 'Jupiter_bedrock_40013_20240617_214449_lying_manny_in_dust_seq'
master_df = pd.read_csv(os.path.join(data_dir, dataset, 'master_annotations.csv'))
master_df = master_df.sort_values('collected_on')

pred_root_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model/'
model = '9_2_unofficial_rev1_lying_down_sitting_25_ep_prod_weights_10_lo_10_tr_br_dr_p2_v_p1_m_p1_u_dust_0624'
pred_dir = os.path.join(pred_root_dir, model, dataset)
dust_df = pd.read_csv(os.path.join(pred_dir, 'dust_ratio.csv'))
dust_df = master_df[['unique_id', 'collected_on']].merge(dust_df, on='unique_id')
save_dir = pred_dir

# # save as a single video
# print(master_df.shape)
# video_name = os.path.join(save_dir, 'pred.mp4')
# ids = master_df.image_id.to_list()
# create_video(ids, pred_dir, video_name, fps=3)

# break into sequences
seq_dfs = get_sequences(master_df, interval=60, per_camera=False, per_camera_pair=True)
print(len(master_df), len(seq_dfs))
video_dir = os.path.join(pred_dir, 'videos')
os.makedirs(video_dir, exist_ok=True)
# cut_off = {1: [0.93, 1.0], 3: [0.5, 1.0], 5: [0.25, 0.75], 9: [0, 1]}
# cut_off = {7: [0.93, 1.0]}
for si, seq_df in enumerate(seq_dfs):
    if len(seq_df) < 10:
        continue
    # if si in cut_off:
    #     cut1, cut2 = cut_off[si]
    #     n = len(seq_df)
    #     seq_df = seq_df.iloc[int(n*cut1): int(n*cut2)]
    # else:
    #     continue
        
    name = seq_df.iloc[0].collected_on
    camera = seq_df.iloc[0].unique_id[-7:]
    print(si, name, camera, len(seq_df))
    
    # create video
    # video_name = os.path.join(video_dir, str(si).zfill(3)+'.mp4')
    video_name = os.path.join(video_dir, f'{camera}_{si}.mp4')
    ids = seq_df.unique_id.to_list()
    create_video(ids, pred_dir, video_name, fps=3, txt_df=dust_df)

/home/li.yu/code/notebooks/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk_df['camera_pair'] = chunk_df['unique_id'].apply(lambda s: s[-7:])
  2%|▏         | 3/195 [00:00<00:06, 28.85it/s]

585 3
0 2024-06-17T21:44:20.065000 T01_T03 195
402 3136 3


  1%|          | 2/195 [00:00<00:09, 19.64it/s]

total 195 used 195
1 2024-06-17T21:44:20.065000 T02_T03 195
580 3137 3


  2%|▏         | 3/195 [00:00<00:06, 28.84it/s]

total 195 used 195
2 2024-06-17T21:44:20.065000 T02_T04 195
402 3136 3


100%|██████████| 195/195 [00:06<00:00, 30.12it/s]

total 195 used 195


### Create video from PP artifacts

In [16]:
def read_from_pp_artifacts(data_dir, df_row):
    data_path = os.path.join(data_dir, df_row.stereo_pipeline_npz_save_path)
    img = np.load(data_path)['left']
    img_norm = normalize_image(img, df_row.hdr_mode if 'hdr_mode' in df_row else True)
    return cv2.cvtColor((img_norm * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)

# def add_text(frame, txt_row):
#     frame = cv2.putText(frame, f'Collected on: {txt_row.collected_on}', 
#                         (40,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
#     return frame

def add_text(frame, txt):
    frame = cv2.putText(frame, txt, 
                        (40,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
    return frame

def add_texts(frame, txts: list):
    txt_pw, txt_ph = 10, 25
    for i, txt in enumerate(txts):
        if 'false_positive' in txt:
            frame = cv2.putText(frame, txt, 
                                (txt_pw, txt_ph+i*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2, cv2.LINE_AA)
        else:
            frame = cv2.putText(frame, txt, 
                                (txt_pw, txt_ph+i*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
    return frame

In [5]:
# data_root_dir = '/data/jupiter/li.yu/data'
data_root_dir = '/data/jupiter/datasets/'
# dataset = 'mannequin_in_dust_v1'
# dataset = 'Jupiter_human_on_path_3_fn_sequence'
dataset = 'halo_missed_lo_rock_0509_stereo'
data_dir = os.path.join(data_root_dir, dataset)

df = pd.read_csv(os.path.join(data_dir, 'master_annotations.csv'))
seq_dfs = get_sequences(df, interval=5, per_camera=True)
print(df.shape, len(seq_dfs))

/home/li.yu/anaconda3/envs/pytorchlightning/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (37,46,58,60,74,86,101,103,106,116,118) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(23023, 150) 432


In [6]:
bags = [
["05_09_2024_18_16_02", "05_09_2024_18_17_02"],
["05_09_2024_18_17_01", "05_09_2024_18_18_01"],
["05_09_2024_18_17_48", "05_09_2024_18_18_48"],
["05_09_2024_18_18_45", "05_09_2024_18_19_45"],
["05_09_2024_18_21_31", "05_09_2024_18_22_31"],
["05_09_2024_18_22_26", "05_09_2024_18_23_26"],
["05_09_2024_18_23_12", "05_09_2024_18_24_12"],
["05_09_2024_18_24_15", "05_09_2024_18_25_15"],
["05_09_2024_20_16_20", "05_09_2024_20_17_20"],
["05_09_2024_20_18_38", "05_09_2024_20_19_38"],
["05_09_2024_20_20_28", "05_09_2024_20_21_28"],
]
bags = [[bag[0][11:].replace('_', ':'), bag[1][11:].replace('_', ':')] for bag in bags]
df['collected_hms'] = df['collected_on'].apply(lambda t: t[11:])
df_oi = []
for bag in bags:
    sub_df = df[(df.collected_hms >= bag[0]) & (df.collected_hms < bag[1])]
    print(bag, len(sub_df))
    df_oi.append(sub_df)
df_oi = pd.concat(df_oi, ignore_index=True).drop_duplicates()
print(df_oi.shape)

seq_dfs = get_sequences(df_oi)
print(df_oi.shape, len(seq_dfs))

['18:16:02', '18:17:02'] 606
['18:17:01', '18:18:01'] 750
['18:17:48', '18:18:48'] 498
['18:18:45', '18:19:45'] 354
['18:21:31', '18:22:31'] 732
['18:22:26', '18:23:26'] 900
['18:23:12', '18:24:12'] 426
['18:24:15', '18:25:15'] 432
['20:16:20', '20:17:20'] 930
['20:18:38', '20:19:38'] 850
['20:20:28', '20:21:28'] 1200
(6568, 151)
(6568, 151) 2


In [8]:
pred_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model'
models = [
    'ds_v8_1_nextvit_small_openimages_with_rev1_train_human_test_using_random_val_mhc_20_epochs_finetune_rev1_lr',
    'v81_80k_maxfov_wn_ft_kore_0430'
]
suffix_list = ['_mhc_depth0125', '']
pred_dfs = [pd.read_csv(os.path.join(pred_dir, model, dataset+suffix, 'output.csv')) for model,suffix in zip(models, suffix_list)]
pred_dfs[0].head(1)

,unique_id,id,camera_location,operation_time,special_notes,jdb_s3_path,result_state,result_human_state,result_vehicle_state,min_pixels_threshold,features,n_gt_human_pixels,gt_human_depth,n_pred_human_pixels,pred_human_depth
0,663ec898321f043ed7ad8a62_T02_T03,663ec898321f043ed7ad8a62,T02,daytime,NaN,NaN,true_negative,true_negative,true_negative,108,"{""large_object_pixels"": 688, ""large_object_min...",0,1000,0,1000


In [18]:
def get_pred_texts(pred_dfs, unique_id, text_prefix):
    texts = []
    for pred_df, prefix in zip(pred_dfs, text_prefix):
        pred_row = pred_df[pred_df.unique_id == unique_id].iloc[0]
        texts.append(f'{prefix}: {pred_row.result_state}')
    return texts

def create_video_from_pp_add_text(seq_df, camera_pairs, pred_dfs, H, W, data_dir, video_dir_name):
    cam_dfs = [seq_df[seq_df.unique_id.str.endswith(c)] for c in camera_pairs]
    cam_dfs = [cdf.sort_values('collected_on', ignore_index=True) for cdf in cam_dfs]
    min_len = min(len(cdf) for cdf in cam_dfs)
    cam_dfs = [cdf.iloc[:min_len] for cdf in cam_dfs]
    print([len(cdf) for cdf in cam_dfs])

    # .avi MJPG,  .mp4 MP4V
    os.makedirs(os.path.join(data_dir, video_dir_name), exist_ok=True)
    video_name = os.path.join(data_dir, f'{video_dir_name}/{seq_df.iloc[0].collected_on}.mp4')

    # print(f'{H} rows, {W} cols out of {len(camera_pairs)} camera_pairs')
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), 3, (768*W,512*H), isColor=True)

    for ii in tqdm(range(min_len)):
        # try:
        canvas = np.zeros((512*H, 768*W, 3), dtype=np.uint8)
        for ci in range(len(cam_dfs)):
            row = cam_dfs[ci].iloc[ii]
            frame = read_from_pp_artifacts(data_dir, row)
            texts = [f'{row.collected_on}'] + get_pred_texts(pred_dfs, row.unique_id, ['MHC pred', 'MAXFOV pred'])
            # print(texts)
            frame = add_texts(frame, texts)
            fi, fj = ci // W, ci % W
            canvas[512*fi:512*(fi+1), 768*fj:768*fj+frame.shape[1]] = frame
        video.write(canvas)
        # except:
        #     print(f'{ii}th image read failed')

    # cv2.destroyAllWindows()
    video.release()

front_pairs = ['T02_T04', 'T02_T03', 'T01_T03']
left_pass_pairs = ['T02_T04', 'T02_T03', 'T01_T03', 'T13_T15', 'T14_T15', 'T14_T16']
right_pass_pairs = ['T02_T04', 'T02_T03', 'T01_T03', 'T06_T08', 'T05_T07', 'T06_T07']
# left_pass_pairs = ['T09_T11', 'T14_T16', 'T14_T15', 'T13_T15']
# right_pass_pairs = ['T05_T07', 'T10_T12', 'T06_T08', 'T06_T07']
H = 1  # number of image rows
W = 3  # number of image cols
create_video_from_pp_add_text(seq_dfs[1], front_pairs, pred_dfs, H, W, data_dir, 'videos_front_pass')

  0%|          | 1/248 [00:00<00:29,  8.30it/s]

[248, 248, 248]


100%|██████████| 248/248 [00:30<00:00,  8.21it/s]


In [36]:
df["camera_pair"] = df["unique_id"].apply(lambda t: t[-7:])
df["camera_pair"].unique()

array(['T01_T03', 'T02_T03', 'T02_T04', 'T05_T07', 'T06_T07', 'T06_T08',
       'T09_T11', 'T10_T12', 'T10_T11', 'T13_T15', 'T14_T16', 'T14_T15'],
      dtype=object)

### Read frame and add prediction results

In [20]:
# compare BRT model pred and CenterTrack pred
# pred_csv = '/data/jupiter/li.yu/exps/driveable_terrain_model/v188_58d_rak_local_fine_tversky11_sum_image_normT_prod5_airdyn_r3a8_s30/mannequin_in_dust_v1/output.csv'
pred_csv = '/data/jupiter/li.yu/exps/driveable_terrain_model/v188_58d_rak_local_fine_tversky11_sum_image_normT_prod5_airdyn_r3a8_s30/Jupiter_human_on_path_3_fn_sequence/output.csv'
pred_df = pd.read_csv(pred_csv)
print(pred_df.shape)

(208, 11)


In [11]:
video_with_pred_dir = os.path.join(data_root_dir, dataset, 'videos_with_pred')
# pred_sequence_dir = '/home/li.yu/code/CenterTrack/results/2023-07-08T01:37:09.798000_front-center_15'
pred_sequence_dir = '/home/li.yu/code/CenterTrack/results/brt50000/nopreimg_noprehm/2023-07-08T01:37:09.798000_front-center_15'
os.makedirs(video_with_pred_dir, exist_ok=True)
height, width = 512, 1024

for si, seq_df in enumerate(seq_dfs):
    if len(seq_df) < 10 or si != 15:
        continue
    name = seq_df.iloc[0].collected_on
    camera = seq_df.iloc[0].camera_location[:-5]
    print(si, name, len(seq_df))
    
    # merge pred from BRT model
    seq_df = seq_df.drop(columns=['state']).merge(pred_df[['id', 'state', 'human_state']], on='id')

    # create video
    video_name = os.path.join(video_with_pred_dir, f'{name}_{camera}_{si}_nopreimg_noprehm.mp4')
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), 3, (width,height), isColor=True)
    fi = 0
    for _, df_row in seq_df.iterrows():
        frame = cv2.imread(os.path.join(pred_sequence_dir, str(fi).zfill(3)+'_'+df_row.id+'.png'))
        frame = cv2.putText(frame, f'BRT Seg Pred: {df_row.state}, Strict: {df_row.human_state}', 
                            (40,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
        video.write(frame)
        fi += 1
    cv2.destroyAllWindows()
    video.release()

    # break


15 2023-07-08T01:37:09.798000 153


### read from video and prediction results to each frame

In [23]:
video_dir = '/home/li.yu/code/CenterTrack/results/'
old_video_name = 'brt50000_2022-11-10T23:19:10.901000_side-right_2_preimg.mp4'
new_video_name = 'brt50000_2022-11-10T23:19:10.901000_side-right_2_preimg_withbrtpred.mp4'
height, width = 512, 1024

for si, seq_df in enumerate(seq_dfs):
    if len(seq_df) < 10 or si != 2:
        continue
    name = seq_df.iloc[0].collected_on
    camera = seq_df.iloc[0].camera_location[:-5]
    print(si, name, len(seq_df))
    
    # merge pred from BRT model
    seq_df = seq_df.drop(columns=['state']).merge(pred_df[['id', 'state', 'human_state']], on='id')

    # read video
    cam = cv2.VideoCapture(os.path.join(video_dir, old_video_name))

    # create video
    video_name = os.path.join(video_dir, new_video_name)
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), 3, (width,height), isColor=True)
    for _, df_row in seq_df.iterrows():
        _, frame = cam.read()
        frame = cv2.putText(frame, f'BRT Seg Pred: {df_row.state}, Strict: {df_row.human_state}', 
                            (40,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
        video.write(frame)
    cv2.destroyAllWindows()
    video.release()

    # break


2 2022-11-10T23:19:10.901000 88
